In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib import image
import os

In [11]:
def image_split(dictionary, overlap, size):
    
   indices = np.arange(0, (5000 - (size - overlap)), size - overlap)
   files = os.listdir(dictionary)
   
   for file in files:
        img = cv2.imread(f'{dictionary}/{file}')
        for x in indices:
            for y in indices:
                split = img[x:(x + size), y:(y + size), :]
                array = np.array(split)
                cv2.imwrite(f'{dictionary}/../simple_slice/{file}_{x}_{y}.png', array)    

In [13]:
image_split('simple_data/single_img/images', 4, 256)

In [16]:
def mask_split(dictionary, overlap, size):
    
   indices = np.arange(0, (5000 - (size - overlap)), size - overlap)
   files = os.listdir(dictionary)
   
   for file in files:
        img = cv2.imread(f'{dictionary}/{file}')
        for x in indices:
            for y in indices:
                split = img[x:(x + size), y:(y + size), :]
                array = np.array(split)
                cv2.imwrite(f'{dictionary}/../mask_slice/{file}_{x}_{y}.png', array)    

In [17]:
mask_split('simple_data/single_img/gt', 4, 256)

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.utils import image_dataset_from_directory

2022-11-29 08:13:51.081248: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/areisdorf/.pyenv/versions/3.8.12/envs/solar/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-29 08:13:51.081298: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [12]:
def simple_unet_model():

    #Input
    height = 256
    width = 256
    channels = 3
    inputs = Input((height, width, channels))
    inputs = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255

    #Encoder where we are extracting the features
    
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = Dropout(0.1)(convolution1) #other option is to do batch normalization
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
    
    conv_block_filters = [16, 32, 64, 128, 256]
    
    for block in conv_block_filters:
        
        conv = Conv2D(filter, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
        conv = BatchNormalization()(conv)
        conv = Conv2D(filter, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
        conv = MaxPooling2D((2, 2))(conv)
        

    
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
    convolution1 = Dropout(0.1)(convolution1) #other option is to do batch normalization
    convolution1 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution1)
    pooling1 = MaxPooling2D((2, 2))(convolution1)
    
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(pooling1)
    convolution2 = Dropout(0.1)(convolution2)
    convolution2 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution2)
    pooling2 = MaxPooling2D((2, 2))(convolution2)
     
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pooling2)
    convolution3 = Dropout(0.2)(convolution3)
    convolution3 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution3)
    pooling3 = MaxPooling2D((2, 2))(convolution3)
     
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(pooling3)
    convolution4 = Dropout(0.2)(convolution4)
    convolution4 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution4)
    pooling4 = MaxPooling2D(pool_size=(2, 2))(convolution4)
     
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(pooling4)
    convolution5 = Dropout(0.3)(convolution5)
    convolution5 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same')(convolution5)
    
    #Decoder where we are indicating to the model the precise location of the features 
    transconv6 = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution5)
    transconv6 = concatenate([transconv6, convolution4])
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(transconv6)
    convolution6 = Dropout(0.2)(convolution6)
    convolution6 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(convolution6)
     
    transconv7 = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution6)
    transconv7 = concatenate([transconv7, convolution3])
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(transconv7)
    convolution7 = Dropout(0.2)(convolution7)
    convolution7 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(convolution7)
     
    transconv8 = Conv2DTranspose(32, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution7)
    transconv8 = concatenate([transconv8, convolution2])
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(transconv8)
    convolution8 = Dropout(0.1)(convolution8)
    convolution8 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(convolution8)
     
    transconv9 = Conv2DTranspose(16, kernel_size=(2, 2), strides=(2, 2), padding='same')(convolution8)
    transconv9 = concatenate([transconv9, convolution1], axis=3)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(transconv9)
    convolution9 = Dropout(0.1)(convolution9)
    convolution9 = Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(convolution9)
     
    outputs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(convolution9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    jaccard = MeanIoU(2)
    
    #loss options include: binary_crossentropy, IoU Loss (Jaccard Index), dice coefficient
    model.compile(optimizer='adam', loss=jaccard, metrics=['accuracy']) 
    
    model.summary()
    
    return model

In [33]:
sliced_image_directory = '/home/areisdorf/code/areisdorf45/solar_panel_segmentation/notebooks/simple_data/single_img/dataset'

In [34]:
images = image_dataset_from_directory(sliced_image_directory)

Found 722 files belonging to 2 classes.


In [2]:
individual_slice = '/home/areisdorf/code/areisdorf45/solar_panel_segmentation/notebooks/simple_data/single_img/simple_slice/austin1.tif_0_504.png'
individual_mask = '/home/areisdorf/code/areisdorf45/solar_panel_segmentation/notebooks/simple_data/single_img/mask_slice/austin1gt.tif_0_504.png'

In [20]:
training_img = cv2.imread(individual_slice)
test_img = cv2.imread(individual_mask)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [13]:
model = simple_unet_model()

NameError: name 'convolution9' is not defined

In [11]:
model.fit(training_img, test_img, epochs=2)

Epoch 1/2


ValueError: in user code:

    File "/home/areisdorf/.pyenv/versions/3.8.12/envs/solar/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/areisdorf/.pyenv/versions/3.8.12/envs/solar/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/areisdorf/.pyenv/versions/3.8.12/envs/solar/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/areisdorf/.pyenv/versions/3.8.12/envs/solar/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/areisdorf/.pyenv/versions/3.8.12/envs/solar/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
  